# 0.0 Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
import inflection

## 0.1 Helper Functions

## 0.2 Loading data

In [2]:
#dataset que contém dados das vendas
df_sales_raw = pd.read_csv('../data/train.csv', low_memory=False)

#dataset que contém dados das lojas
df_stores_raw = pd.read_csv('../data/store.csv', low_memory=False)

#dataset final
df_raw = pd.merge(df_sales_raw, df_stores_raw, how='left', on='Store')

In [3]:
df_sales_raw.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [4]:
df_stores_raw.head()

,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [5]:
df_raw.head()

,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,5,2015-07-31,5263,555,1,1,0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,5,2015-07-31,6064,625,1,1,0,1,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,5,2015-07-31,8314,821,1,1,0,1,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,5,2015-07-31,13995,1498,1,1,0,1,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,5,2015-07-31,4822,559,1,1,0,1,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


# 1.0 Descrição dos dados

#### Qual é a motivação por trás dessa tarefa?

In [6]:
df1 = df_raw.copy()

## 1.1 Rename Columns

Descobrir o quão desafiador é o problema.

* É preciso saber a quantidade de dados:
    * eu tenho recursos corretos para trabalhar?
    * preciso de servidores? clusters? ferramentas para trabalhar com esses dados, como spark e hadoop?
    
* Tipos de variáveis:
    * quais os tipos de variáveis? % de numéricas, categóricas e temporais.
    
* Quantidade de dados faltantes:
    * qual o volume de dados NA?
    * qual a razão de faltar dados? é o sistema? input manual?
    
* Resumo geral dos dados:
    * estatística descritiva - importante para ter uma noção dos dados. dos limites máximos e mínimos..

É importante para criar um padrão nos nomes da coluna, além de renomear colunas que possuem nomes que não fazem muito sentido ou que são muito extensos. Isso facilita todo o desenvolvimento. Nesse ambiente do Kaggle e de datasets de estudo, as colunas já vem melhor nomeadas. Mas no contexto real nem sempre é assim. Por isso é uma boa prática realizar essa etapa.

In [7]:
#colunas antigas
cols_old = ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 
            'SchoolHoliday', 'StoreType', 'Assortment', 'CompetitionDistance', 'CompetitionOpenSinceMonth',
            'CompetitionOpenSinceYear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval']

#função para alterar o nome das colunas para o formato snakecase
change_to_snakecase = lambda x: inflection.underscore(x)

#aplicando a função nas colunas antigas
new_cols = list(map(change_to_snakecase, cols_old))

#renomeando as colunas
df1.columns = new_cols

#verificando os novos nomes
df1.columns

Index(['store', 'day_of_week', 'date', 'sales', 'customers', 'open', 'promo',
       'state_holiday', 'school_holiday', 'store_type', 'assortment',
       'competition_distance', 'competition_open_since_month',
       'competition_open_since_year', 'promo2', 'promo2_since_week',
       'promo2_since_year', 'promo_interval'],
      dtype='object')

## 1.2 Data dimension

In [8]:
shape = df1.shape
print(f'Number of rows: {shape[0]}')
print(f'Number of columns: {shape[1]}')

Number of rows: 1017209
Number of columns: 18


## 1.3 Data types

In [16]:
#alterando o tipo da coluna date para datetime (estava como object)
df1['date'] = pd.to_datetime(df1['date'])

df1.dtypes

store                                    int64
day_of_week                              int64
date                            datetime64[ns]
sales                                    int64
customers                                int64
open                                     int64
promo                                    int64
state_holiday                           object
school_holiday                           int64
store_type                              object
assortment                              object
competition_distance                   float64
competition_open_since_month           float64
competition_open_since_year            float64
promo2                                   int64
promo2_since_week                      float64
promo2_since_year                      float64
promo_interval                          object
dtype: object

## 1.4 Verificação de dados faltantes

In [17]:
df1.isna().sum()

store                                0
day_of_week                          0
date                                 0
sales                                0
customers                            0
open                                 0
promo                                0
state_holiday                        0
school_holiday                       0
store_type                           0
assortment                           0
competition_distance              2642
competition_open_since_month    323348
competition_open_since_year     323348
promo2                               0
promo2_since_week               508031
promo2_since_year               508031
promo_interval                  508031
dtype: int64

Existem basicamente três maneiras para lidar com dados faltantes:
1. Eliminar as linhas que possuem NA
    * Vantagem: rápido e fácil
    * Desvantagem: pode desperdiçar dados
    
    
2. Substituir os faltantes por métricas (média, mediana...) ou por previsão de algoritmos


3. Conhecendo do negócio e entendendo por que esses NAs estão aí


## 1.5 Substituição dos dados faltantes